In [2]:
import json
from datetime import date, timedelta
import pandas as pd

try:
    import requests
    HAS_REQUESTS = True
except ImportError:
    import urllib.request
    HAS_REQUESTS = False


def daterange(start_date: date, end_date: date):
    
    d = start_date
    while d <= end_date:
        yield d
        d += timedelta(days=1)

def build_kalshi_url(kind: str, d: date) -> str:
    """
    kind: 'market_data' or 'trade_data'
    """
    ds = d.strftime("%Y-%m-%d")
    return f"https://kalshi-public-docs.s3.amazonaws.com/reporting/{kind}_{ds}.json"

def fetch_json(url: str, timeout: int = 30):
    """Fetch JSON from URL, return parsed object, or None if not found / error."""

    try:
        if HAS_REQUESTS:
            r = requests.get(url, timeout=timeout)
            if r.status_code != 200:
                return None
            return r.json()
        else:
            with urllib.request.urlopen(url, timeout=timeout) as resp:
                if resp.status != 200:
                    return None
                return json.loads(resp.read().decode("utf-8"))
    except Exception:
        return None
    

def json_to_frames(obj):

    """
    converts kalshi json data to one or more dataframes
    """

    frames = {}

    if obj is None:
        return frames
    
    if isinstance(obj, list):
        frames["root"] = pd.json_normalize(obj)
        return frames
    if isinstance(obj, dict):
        found_list_tables = False
        for k, v in obj.items():
            if isinstance(v, list) and len(v) > 0 and isinstance(v[0], dict):
                frames[k] = pd.json_normalize(v)
                found_list_tables = True

        if found_list_tables:
            return frames
        
        frames["root"] = pd.json_normalize(obj)
        return frames
    
    frames["root"] = pd.DataFrame({"raw": [str(obj)]})
    return frames


In [3]:
#---CONFIG---
KIND = "market_data"
START_DATE = date(2025, 9, 17)
END_DATE = date(2025, 12, 7)

TICKERS = ["KXNCAAFPLAYOFF-25"]

In [ ]:
raw_daily_frames = []
missing_days = []

for d in daterange(START_DATE, END_DATE):
    url = build_kalshi_url(KIND, d)
    obj = fetch_json(url)

    if obj is None:
        missing_days.append(d)
        continue

    frames = json_to_frames(obj)



    if len(frames) == 0:
        missing_days.append(d)
        continue

    best_key = max(frames.keys(), key=lambda k: len(frames[k]))
    df = frames[best_key].copy()
    df["as_of_date"] = pd.to_datetime(d)

    raw_daily_frames.append(df)

all_data = pd.concat(raw_daily_frames, ignore_index=True) if raw_daily_frames else pd.DataFrame()

print("Rows:", len(all_data))
print("Cols:", len(all_data.columns))
print("Missing days:", len(missing_days))

if missing_days:
    print("First few missing:", missing_days[:10])

In [ ]:
all_data.columns.tolist()[:60]
all_data.head(3) 
   

In [ ]:
possible_ticker_cols = ["ticker_name", "ticker", "market_ticker", "event_ticker"]

ticker_col = next((c for c in possible_ticker_cols if c in all_data.columns), None)
if ticker_col is None:
    raise ValueError(
        f"Couldn't find a ticker column. Looked for {possible_ticker_cols}. "
        f"Available columns include: {all_data.columns.tolist()[:50]}"
    )

filtered = all_data[all_data[ticker_col].isin(TICKERS)].copy()

print("Ticker column used:", ticker_col)
print("Filtered rows:", len(filtered))
filtered.head(3)


In [ ]:
out_path = f"kalshi_{KIND}_{START_DATE}_{END_DATE}_filtered.csv".replace(":", "-")
filtered.to_csv(out_path, index=False)
out_path
